<a href="https://colab.research.google.com/github/konkuk-gaegul/3rd-Team-Project/blob/main/%EA%B2%AC%EC%A2%85_%ED%86%B5%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전처리 설명
- 통합 자료 Set 6와 10을 활용한다.
    - set 6  : 대비향상(YUV 적용), 선명도
    - set 10 : 이미지크기 선명도 배경제거(높이180) 선명하게 대비높이기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time

import copy
import os, shutil
import matplotlib as plt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

In [ ]:
base_dir = '/home/tutor/안재형/dog_pic/통합/set_6'

# # 훈련셋, 검증셋, 테스트셋을 미리 분할하기 위한 폴더 경로
train_path = os.path.join(base_dir, 'train')
os.mkdir( train_path)

test_path = os.path.join(base_dir, 'test')
os.mkdir( test_path)


# # train 정상 폴더
train_nor_path = os.path.join(train_path, 'nor')
os.mkdir( train_nor_path)

# # train 비만 폴더
train_fat_path = os.path.join(train_path, 'fat')
os.mkdir( train_fat_path)

# # test 정상 폴더
test_nor_path = os.path.join(test_path, 'nor')
os.mkdir( test_nor_path)

# # test 비만 폴더
test_fat_path = os.path.join(test_path, 'fat')
os.mkdir( test_fat_path)

base_dir = '/home/tutor/안재형/dog_pic/통합/set_6'

nor_path = os.path.join(base_dir, '전처리_정상')
fat_path = os.path.join(base_dir, '전처리_비만')

nor_list = os.listdir(nor_path)
fat_list = os.listdir(fat_path)

print(len(nor_list))
print(len(fat_list))

# train : 75%, test : 25%
# 정상 이미지 복사
for i in range(len(nor_list)):
    src_path = os.path.join(nor_path, nor_list[i])
    
    if i < len(nor_list)*0.75:
        dst_path = os.path.join(train_nor_path, f'nor_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
    else :
        dst_path = os.path.join(test_nor_path, f'nor_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
                                
# 비만 이미지 복사
for i in range(len(fat_list)):
    src_path = os.path.join(fat_path, fat_list[i])
    
    if i < len(fat_list)*0.75:
        dst_path = os.path.join(train_fat_path, f'fat_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
    else :
        dst_path = os.path.join(test_fat_path, f'fat_{i}.jpg')
        shutil.copyfile( src_path, dst_path )

6505
1239


In [ ]:
print( len( os.listdir(train_nor_path) ) )
print( len( os.listdir(train_fat_path) ) )
print( len( os.listdir(test_nor_path) ) )
print( len( os.listdir(test_fat_path) ) )

4879
930
1626
309


# 학습 1
- 이미지 통합 set_6 활용

In [ ]:
Loss_Accuracy = []

In [ ]:
epochs = [50]

In [ ]:
for epoch in epochs:
    for i in range(6, 7):
        # set 9에서 j 번 째 학습을 의미
        for batch in range(8, 12):
            # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
            # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
            transforms_train = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
            ])

            transforms_test = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

            data_dir = '/home/tutor/안재형/dog_pic/통합'
            data_folder = os.path.join(data_dir, f'set_{i}')
            train_datasets = datasets.ImageFolder(os.path.join(data_folder, 'train'), transforms_train)
            test_datasets = datasets.ImageFolder(os.path.join(data_folder, 'test'), transforms_test)

            train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=True, num_workers=2)
            test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch, shuffle=True, num_workers=2)

    #         print('학습 데이터셋 크기:', len(train_datasets))
    #         print('테스트 데이터셋 크기:', len(test_datasets))

            class_names = train_datasets.classes
    #         print('클래스:', class_names)

            #--------------------------------------------#

            model = models.resnet34(pretrained=True)
            num_features = model.fc.in_features
            # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
            model.fc = nn.Linear(num_features, 2)
            model = model.to(device)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

            #--------------------------------------------#
            print(f'-------------현재 set_{i} batch_szie {batch} epoch {epoch}-------------')
            num_epochs = epoch
            model.train()
            start_time = time.time()

            # 전체 반복(epoch) 수 만큼 반복하며
            for epoch in range(num_epochs):
                running_loss = 0.
                running_corrects = 0

                # 배치 단위로 학습 데이터 불러오기
                for inputs, labels in train_dataloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # 모델에 입력(forward)하고 결과 계산
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(train_datasets)
                epoch_acc = running_corrects / len(train_datasets) * 100.

                # 학습 과정 중에 결과 출력
                print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

            #--------------------------------------------#

            model.eval()
            start_time = time.time()

            with torch.no_grad():
                running_loss = 0.
                running_corrects = 0

                for inputs, labels in test_dataloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
                    print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        #             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

                epoch_loss = running_loss / len(test_datasets)
                epoch_acc = running_corrects / len(test_datasets) * 100.
                print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

                Loss_Accuracy.append([i, batch, epoch_loss, epoch_acc])
                torch.save(model, f'total_set{i}_B{batch}_1.pth')
            # 모든 모델을 저장하지 않고 가장 좋은 성능의 모델을 저장
            if batch == 8:
                Best = [i, batch, epoch_loss, epoch_acc]

            elif epoch_loss < Best[2]:
                # Best 모델보다 Loss가 개선되면 저장
                Best = [i, batch, epoch_loss, epoch_acc]

            print(f'현재 Best는 batch_size : {Best[1]} / Loss : {Best[2]} / Acc : {Best[3]}')
    #         print('--------------------------------------------------')

-------------현재 set_6 batch_szie 8 epoch 50-------------
#0 Loss: 0.3476 Acc: 85.7807% Time: 55.3120s
#1 Loss: 0.2613 Acc: 89.1031% Time: 111.0232s
#2 Loss: 0.2308 Acc: 90.5491% Time: 167.8166s
#3 Loss: 0.1833 Acc: 92.3395% Time: 224.3146s
#4 Loss: 0.1661 Acc: 93.4929% Time: 279.7733s
#5 Loss: 0.1382 Acc: 94.5946% Time: 336.6453s
#6 Loss: 0.1139 Acc: 95.5242% Time: 393.4132s
#7 Loss: 0.1020 Acc: 96.2816% Time: 449.9803s
#8 Loss: 0.0937 Acc: 96.1956% Time: 507.2076s
#9 Loss: 0.0871 Acc: 96.6431% Time: 564.2996s
#10 Loss: 0.0792 Acc: 96.9530% Time: 620.2982s
#11 Loss: 0.0660 Acc: 97.2457% Time: 675.8365s
#12 Loss: 0.0678 Acc: 97.4178% Time: 731.9883s
#13 Loss: 0.0659 Acc: 97.5555% Time: 788.0578s
#14 Loss: 0.0490 Acc: 97.9342% Time: 844.6827s
#15 Loss: 0.0565 Acc: 97.8137% Time: 901.2095s
#16 Loss: 0.0473 Acc: 97.9342% Time: 957.9279s
#17 Loss: 0.0540 Acc: 97.7965% Time: 1013.8978s
#18 Loss: 0.0486 Acc: 97.9515% Time: 1069.9707s
#19 Loss: 0.0506 Acc: 98.0031% Time: 1126.3636s
#20 Loss: 0

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[Test Phase] Loss: 0.5251 Acc: 90.1292% Time: 13.3135s
현재 Best는 batch_size : 8 / Loss : 0.5251310307397267 / Acc : 90.12919616699219
-------------현재 set_6 batch_szie 9 epoch 49-------------
#0 Loss: 0.3508 Acc: 85.5397% Time: 56.5491s
#1 Loss: 0.2556 Acc: 88.8793% Time: 112.0815s
#2 Loss: 0.2133 Acc: 91.3238% Time: 167.8161s
#3 Loss: 0.1859 Acc: 92.4600% Time: 223.0425s
#4 Loss: 0.1480 Acc: 94.1814% Time: 277.7861s
#5 Loss: 0.1345 Acc: 94.8184% Time: 332.5593s
#6 Loss: 0.1141

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[Test Phase] Loss: 0.6046 Acc: 90.7494% Time: 13.1040s
현재 Best는 batch_size : 8 / Loss : 0.5251310307397267 / Acc : 90.12919616699219
-------------현재 set_6 batch_szie 10 epoch 48-------------
#0 Loss: 0.3558 Acc: 85.1954% Time: 54.8153s
#1 Loss: 0.2458 Acc: 89.9294% Time: 109.8077s
#2 Loss: 0.2091 Acc: 91.1689% Time: 164.5589s
#3 Loss: 0.1675 Acc: 93.6650% Time: 219.0827s
#4 Loss: 0.1476 Acc: 94.0609% Time: 274.4322s
#5 Loss: 0.1281 Acc: 95.1110% Time: 329.5435s
#6 Loss: 0.1132 Acc: 95.4725% Time: 384.7644s
#7 Loss: 0.0952 Acc: 96.0923% Time: 439.9284s
#8 Loss: 0.0887 Acc: 96.31

#2 Loss: 0.1974 Acc: 91.5992% Time: 165.0264s
#3 Loss: 0.1728 Acc: 93.1486% Time: 218.4868s
#4 Loss: 0.1358 Acc: 94.4741% Time: 272.6697s
#5 Loss: 0.1236 Acc: 95.0250% Time: 326.4790s
#6 Loss: 0.1143 Acc: 95.6963% Time: 380.7514s
#7 Loss: 0.0941 Acc: 96.1267% Time: 436.2148s
#8 Loss: 0.0777 Acc: 96.9358% Time: 490.6689s
#9 Loss: 0.0777 Acc: 96.8153% Time: 545.8229s
#10 Loss: 0.0606 Acc: 97.7621% Time: 600.9744s
#11 Loss: 0.0681 Acc: 97.0219% Time: 656.9881s
#12 Loss: 0.0649 Acc: 97.3662% Time: 711.4839s
#13 Loss: 0.0561 Acc: 97.7621% Time: 765.8182s
#14 Loss: 0.0423 Acc: 98.3302% Time: 820.2555s
#15 Loss: 0.0417 Acc: 98.2957% Time: 874.8878s
#16 Loss: 0.0409 Acc: 98.2441% Time: 929.2281s
#17 Loss: 0.0354 Acc: 98.3646% Time: 983.9252s
#18 Loss: 0.0352 Acc: 98.6228% Time: 1038.6833s
#19 Loss: 0.0428 Acc: 98.2269% Time: 1093.8090s
#20 Loss: 0.0418 Acc: 98.1236% Time: 1147.9485s
#21 Loss: 0.0317 Acc: 98.4679% Time: 1202.6079s
#22 Loss: 0.0293 Acc: 98.3990% Time: 1257.7351s
#23 Loss: 0.0283

In [ ]:
Loss_Accuracy

[[6, 8, 0.5251310307397267, tensor(90.1292, device='cuda:0')],
 [6, 9, 0.6045617199913025, tensor(90.7494, device='cuda:0')],
 [6, 10, 0.5343055593720558, tensor(90.3359, device='cuda:0')],
 [6, 11, 0.5302106495151236, tensor(91.1628, device='cuda:0')]]

## 학습 1 후기
- 최고 성능 : batch_size = 11, Loss = 0.5302, Acc : 91.16%

In [ ]:
epochs = [50]
Loss_Accuracy = []

In [ ]:
for epoch in epochs:
    for i in range(6, 7):
        # set 9에서 j 번 째 학습을 의미
        for batch in range(8, 12):
            # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
            # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
            transforms_train = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
            ])

            transforms_test = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

            data_dir = '/home/tutor/안재형/dog_pic/통합'
            data_folder = os.path.join(data_dir, f'set_{i}')
            train_datasets = datasets.ImageFolder(os.path.join(data_folder, 'train'), transforms_train)
            test_datasets = datasets.ImageFolder(os.path.join(data_folder, 'test'), transforms_test)

            train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=True, num_workers=2)
            test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch, shuffle=True, num_workers=2)

    #         print('학습 데이터셋 크기:', len(train_datasets))
    #         print('테스트 데이터셋 크기:', len(test_datasets))

            class_names = train_datasets.classes
    #         print('클래스:', class_names)

            #--------------------------------------------#

            model = models.resnet34(pretrained=True)
            num_features = model.fc.in_features
            # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
            model.fc = nn.Linear(num_features, 2)
            model = model.to(device)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

            #--------------------------------------------#
            num_epochs = epoch
            print(f'-------------현재 set_{i} batch_szie {batch} epoch {num_epochs}-------------')
            model.train()
            start_time = time.time()

            # 전체 반복(epoch) 수 만큼 반복하며
            for epoch in range(num_epochs):
                running_loss = 0.
                running_corrects = 0

                # 배치 단위로 학습 데이터 불러오기
                for inputs, labels in train_dataloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # 모델에 입력(forward)하고 결과 계산
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(train_datasets)
                epoch_acc = running_corrects / len(train_datasets) * 100.

                # 학습 과정 중에 결과 출력
                print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

            #--------------------------------------------#

            model.eval()
            start_time = time.time()

            with torch.no_grad():
                running_loss = 0.
                running_corrects = 0

                for inputs, labels in test_dataloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
                    print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        #             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

                epoch_loss = running_loss / len(test_datasets)
                epoch_acc = running_corrects / len(test_datasets) * 100.
                print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

                Loss_Accuracy.append([i, batch, epoch_loss, epoch_acc])
                torch.save(model, f'total_set{i}_B{batch}_1.pth')
            # 모든 모델을 저장하지 않고 가장 좋은 성능의 모델을 저장
            if batch == 8 and num_epochs == 75:
                Best = [i, batch, epoch_loss, epoch_acc]

            elif epoch_loss < Best[2]:
                # Best 모델보다 Loss가 개선되면 저장
                Best = [i, batch, epoch_loss, epoch_acc]

            print(f'현재 Best는 batch_size : {Best[1]} / Loss : {Best[2]} / Acc : {Best[3]}')
    #         print('--------------------------------------------------')

-------------현재 set_6 batch_szie 8 epoch 75-------------
#0 Loss: 0.3508 Acc: 85.1610% Time: 55.4105s
#1 Loss: 0.2695 Acc: 88.7244% Time: 111.3574s
#2 Loss: 0.2198 Acc: 90.7729% Time: 166.9449s
#3 Loss: 0.1758 Acc: 92.6838% Time: 222.5391s
#4 Loss: 0.1617 Acc: 93.3379% Time: 280.5816s
#5 Loss: 0.1369 Acc: 94.4397% Time: 336.6097s
#6 Loss: 0.1290 Acc: 94.9905% Time: 392.5405s
#7 Loss: 0.1072 Acc: 95.7996% Time: 449.2961s
#8 Loss: 0.1009 Acc: 96.1439% Time: 504.9654s
#9 Loss: 0.0874 Acc: 96.7292% Time: 562.2595s
#10 Loss: 0.0905 Acc: 96.6259% Time: 618.1027s
#11 Loss: 0.0676 Acc: 97.2629% Time: 674.4145s
#12 Loss: 0.0588 Acc: 97.5555% Time: 730.5993s
#13 Loss: 0.0594 Acc: 97.4350% Time: 787.2073s
#14 Loss: 0.0580 Acc: 97.6932% Time: 842.6767s
#15 Loss: 0.0525 Acc: 97.7104% Time: 897.7647s
#16 Loss: 0.0528 Acc: 97.8826% Time: 953.4650s
#17 Loss: 0.0473 Acc: 98.1236% Time: 1009.1484s
#18 Loss: 0.0405 Acc: 98.2613% Time: 1064.4165s
#19 Loss: 0.0361 Acc: 98.3646% Time: 1120.2433s
#20 Loss: 0

[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor]

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor]

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat]

[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor]

# 학습 2
- 이미지 통합 set 10

In [ ]:
%cd /home/tutor/안재형/saved_files/통합

/home/tutor/안재형/saved_files/통합


In [ ]:
base_dir = '/home/tutor/안재형/dog_pic/통합/set_10'

# # 훈련셋, 검증셋, 테스트셋을 미리 분할하기 위한 폴더 경로
train_path = os.path.join(base_dir, 'train')
os.mkdir( train_path)

test_path = os.path.join(base_dir, 'test')
os.mkdir( test_path)


# # train 정상 폴더
train_nor_path = os.path.join(train_path, 'nor')
os.mkdir( train_nor_path)

# # train 비만 폴더
train_fat_path = os.path.join(train_path, 'fat')
os.mkdir( train_fat_path)

# # test 정상 폴더
test_nor_path = os.path.join(test_path, 'nor')
os.mkdir( test_nor_path)

# # test 비만 폴더
test_fat_path = os.path.join(test_path, 'fat')
os.mkdir( test_fat_path)

base_dir = '/home/tutor/안재형/dog_pic/통합/set_6'

nor_path = os.path.join(base_dir, '전처리_정상')
fat_path = os.path.join(base_dir, '전처리_비만')

nor_list = os.listdir(nor_path)
fat_list = os.listdir(fat_path)

print(len(nor_list))
print(len(fat_list))

# train : 75%, test : 25%
# 정상 이미지 복사
for i in range(len(nor_list)):
    src_path = os.path.join(nor_path, nor_list[i])
    
    if i < len(nor_list)*0.75:
        dst_path = os.path.join(train_nor_path, f'nor_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
    else :
        dst_path = os.path.join(test_nor_path, f'nor_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
                                
# 비만 이미지 복사
for i in range(len(fat_list)):
    src_path = os.path.join(fat_path, fat_list[i])
    
    if i < len(fat_list)*0.75:
        dst_path = os.path.join(train_fat_path, f'fat_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
    else :
        dst_path = os.path.join(test_fat_path, f'fat_{i}.jpg')
        shutil.copyfile( src_path, dst_path )

6505
1239


In [ ]:
epochs = [50]
Loss_Accuracy = []

In [ ]:
for epoch in epochs:
    for i in range(10, 11):
        # set 9에서 j 번 째 학습을 의미
        for batch in range(5, 12):
            # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
            # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
            transforms_train = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
            ])

            transforms_test = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

            data_dir = '/home/tutor/안재형/dog_pic/통합'
            data_folder = os.path.join(data_dir, f'set_{i}')
            train_datasets = datasets.ImageFolder(os.path.join(data_folder, 'train'), transforms_train)
            test_datasets = datasets.ImageFolder(os.path.join(data_folder, 'test'), transforms_test)

            train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=True, num_workers=2)
            test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch, shuffle=True, num_workers=2)

    #         print('학습 데이터셋 크기:', len(train_datasets))
    #         print('테스트 데이터셋 크기:', len(test_datasets))

            class_names = train_datasets.classes
    #         print('클래스:', class_names)

            #--------------------------------------------#

            model = models.resnet34(pretrained=True)
            num_features = model.fc.in_features
            # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
            model.fc = nn.Linear(num_features, 2)
            model = model.to(device)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

            #--------------------------------------------#
            num_epochs = epoch
            print(f'-------------현재 set_{i} batch_szie {batch} epoch {epoch}-------------')
            model.train()
            start_time = time.time()

            # 전체 반복(epoch) 수 만큼 반복하며
            for epoch in range(num_epochs):
                running_loss = 0.
                running_corrects = 0

                # 배치 단위로 학습 데이터 불러오기
                for inputs, labels in train_dataloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # 모델에 입력(forward)하고 결과 계산
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(train_datasets)
                epoch_acc = running_corrects / len(train_datasets) * 100.

                # 학습 과정 중에 결과 출력
                print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

            #--------------------------------------------#

            model.eval()
            start_time = time.time()

            with torch.no_grad():
                running_loss = 0.
                running_corrects = 0

                for inputs, labels in test_dataloader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
                    print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        #             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

                epoch_loss = running_loss / len(test_datasets)
                epoch_acc = running_corrects / len(test_datasets) * 100.
                print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

                Loss_Accuracy.append([i, batch, epoch_loss, epoch_acc])
                torch.save(model, f'total_set{i}_B{batch}_3.pth')
            # 모든 모델을 저장하지 않고 가장 좋은 성능의 모델을 저장
            if batch == 5 :
                Best = [i, batch, epoch_loss, epoch_acc]

            elif epoch_loss < Best[2]:
                # Best 모델보다 Loss가 개선되면 저장
                Best = [i, batch, epoch_loss, epoch_acc]

            print(f'현재 Best는 batch_size : {Best[1]} / Loss : {Best[2]} / Acc : {Best[3]}')
    #         print('--------------------------------------------------')

-------------현재 set_10 batch_szie 5 epoch 50-------------
#0 Loss: 0.3987 Acc: 83.9387% Time: 57.4090s
#1 Loss: 0.2994 Acc: 87.7432% Time: 115.4074s
#2 Loss: 0.2625 Acc: 89.7401% Time: 173.8927s
#3 Loss: 0.2221 Acc: 91.4271% Time: 232.0733s
#4 Loss: 0.1862 Acc: 92.7871% Time: 291.0009s
#5 Loss: 0.1715 Acc: 93.2174% Time: 349.4843s
#6 Loss: 0.1491 Acc: 94.4397% Time: 407.0796s
#7 Loss: 0.1370 Acc: 95.0250% Time: 465.0219s
#8 Loss: 0.1203 Acc: 95.4209% Time: 523.6276s
#9 Loss: 0.1145 Acc: 95.8341% Time: 582.3829s
#10 Loss: 0.0948 Acc: 96.4710% Time: 640.3343s
#11 Loss: 0.0873 Acc: 96.7464% Time: 699.9964s
#12 Loss: 0.0814 Acc: 96.9186% Time: 758.2104s
#13 Loss: 0.0749 Acc: 97.1940% Time: 817.4970s
#14 Loss: 0.0745 Acc: 97.3662% Time: 875.8997s
#15 Loss: 0.0657 Acc: 97.6416% Time: 934.9927s
#16 Loss: 0.0535 Acc: 98.1236% Time: 992.8011s
#17 Loss: 0.0632 Acc: 97.9170% Time: 1050.4297s
#18 Loss: 0.0534 Acc: 97.8482% Time: 1108.0232s
#19 Loss: 0.0537 Acc: 97.8309% Time: 1166.6833s
#20 Loss: 

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor]

[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor]

#23 Loss: 0.0264 Acc: 98.7261% Time: 1348.8651s
#24 Loss: 0.0306 Acc: 98.6400% Time: 1406.3300s
#25 Loss: 0.0355 Acc: 98.2441% Time: 1462.3588s
#26 Loss: 0.0341 Acc: 98.3474% Time: 1518.3248s
#27 Loss: 0.0372 Acc: 98.4163% Time: 1575.4090s
#28 Loss: 0.0374 Acc: 98.3646% Time: 1631.6272s
#29 Loss: 0.0460 Acc: 98.1408% Time: 1688.2736s
#30 Loss: 0.0379 Acc: 98.3130% Time: 1743.9679s
#31 Loss: 0.0406 Acc: 98.2613% Time: 1800.8163s
#32 Loss: 0.0301 Acc: 98.4335% Time: 1857.4881s
#33 Loss: 0.0309 Acc: 98.6917% Time: 1913.8344s
#34 Loss: 0.0289 Acc: 98.4163% Time: 1970.6956s
#35 Loss: 0.0275 Acc: 98.5195% Time: 2028.0096s
#36 Loss: 0.0236 Acc: 98.7950% Time: 2084.0922s
#37 Loss: 0.0217 Acc: 98.7089% Time: 2140.8661s
#38 Loss: 0.0264 Acc: 98.7261% Time: 2196.2697s
#39 Loss: 0.0257 Acc: 98.5540% Time: 2253.0984s
#40 Loss: 0.0235 Acc: 98.5884% Time: 2308.4852s
#41 Loss: 0.0328 Acc: 98.4679% Time: 2364.3086s
#42 Loss: 0.0309 Acc: 98.4335% Time: 2420.2118s
#43 Loss: 0.0253 Acc: 98.5884% Time: 247

[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[Test Phase] Loss: 0.5298 Acc: 91.1111% Time: 12.2237s
현재 Best는 batch_size : 6 / Loss : 0.48890495240735626 / Acc : 89.0439224243164
-------------현재 set_10 batch_szie 8 epoch 47-------------
#0 Loss: 0.3549 Acc: 85.6602% Time: 57.0087s
#1 Loss: 0.2560 Acc: 89.3785% Time: 113.7089s
#2 Loss: 0.2310 Acc: 90.3081% Time: 168.8449s
#3 Loss: 0.1806 Acc: 92.6149% Time: 225.4252s
#4 Loss: 0.1577 Acc: 94.0265% Time: 281.4237s
#5 Loss: 0.1369 Acc: 94.7840% Time: 338.4787s
#6 Loss: 0.1229 Acc: 95.0938% Time: 396.1598s
#7 Loss: 0.1168 Acc: 95.4209% Time: 453.5545s
#8 Loss: 0.0925 Acc: 96.6259% Time: 509.5667s
#9 Loss: 0.0922 Acc: 96.3677% Time: 566.2172s
#10 Loss: 0.0795 Acc: 96.9702% Time: 622.2023s
#11 Loss: 0.0726 Acc: 97.2629% Time: 678.3561s
#12 Loss: 0.0624 Acc: 97.6588% Time: 734.4491s
#13 Loss: 0.0586 Acc: 97.67

KeyboardInterrupt: 

In [ ]:
Loss_Accuracy

[[10, 5, 0.7275408057156556, tensor(85.9432, device='cuda:0')],
 [10, 6, 0.48890495240735626, tensor(89.0439, device='cuda:0')],
 [10, 7, 0.5297667563251747, tensor(91.1111, device='cuda:0')]]

In [ ]:
Loss_Accuracy = []

In [ ]:
for i in range(10, 11):
    # set 9에서 j 번 째 학습을 의미
    for batch in range(8, 12):
            # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
            # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
        transforms_train = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
            ])

        transforms_test = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

        data_dir = '/home/tutor/안재형/dog_pic/통합'
        data_folder = os.path.join(data_dir, f'set_{i}')
        train_datasets = datasets.ImageFolder(os.path.join(data_folder, 'train'), transforms_train)
        test_datasets = datasets.ImageFolder(os.path.join(data_folder, 'test'), transforms_test)

        train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=True, num_workers=2)
        test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch, shuffle=True, num_workers=2)

    #         print('학습 데이터셋 크기:', len(train_datasets))
    #         print('테스트 데이터셋 크기:', len(test_datasets))

        class_names = train_datasets.classes
    #         print('클래스:', class_names)

            #--------------------------------------------#

        model = models.resnet34(pretrained=True)
        num_features = model.fc.in_features
            # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
        model.fc = nn.Linear(num_features, 2)
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

        #--------------------------------------------#
        num_epochs = 50
        print(f'-------------현재 set_{i} batch_szie {batch} -------------')
        model.train()
        start_time = time.time()

        # 전체 반복(epoch) 수 만큼 반복하며
        for epoch in range(num_epochs):
            running_loss = 0.
            running_corrects = 0

            # 배치 단위로 학습 데이터 불러오기
            for inputs, labels in train_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 모델에 입력(forward)하고 결과 계산
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(train_datasets)
            epoch_acc = running_corrects / len(train_datasets) * 100.

            # 학습 과정 중에 결과 출력
            print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

        #--------------------------------------------#

        model.eval()
        start_time = time.time()

        with torch.no_grad():
            running_loss = 0.
            running_corrects = 0

            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
                print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        #             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

            epoch_loss = running_loss / len(test_datasets)
            epoch_acc = running_corrects / len(test_datasets) * 100.
            print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

            Loss_Accuracy.append([i, batch, epoch_loss, epoch_acc])
            torch.save(model, f'total_set{i}_B{batch}_3.pth')
            # 모든 모델을 저장하지 않고 가장 좋은 성능의 모델을 저장
        if batch == 8 :
            Best = [i, batch, epoch_loss, epoch_acc]

        elif epoch_loss < Best[2]:
            # Best 모델보다 Loss가 개선되면 저장
            Best = [i, batch, epoch_loss, epoch_acc]

        print(f'현재 Best는 batch_size : {Best[1]} / Loss : {Best[2]} / Acc : {Best[3]}')
    #         print('--------------------------------------------------')

-------------현재 set_10 batch_szie 8 -------------
#0 Loss: 0.3499 Acc: 85.0232% Time: 56.9555s
#1 Loss: 0.2705 Acc: 88.8793% Time: 113.7221s
#2 Loss: 0.2127 Acc: 91.3066% Time: 170.7766s
#3 Loss: 0.1824 Acc: 92.6838% Time: 227.1812s
#4 Loss: 0.1550 Acc: 93.6994% Time: 283.4850s
#5 Loss: 0.1390 Acc: 94.4913% Time: 339.5709s
#6 Loss: 0.1247 Acc: 95.2143% Time: 395.2160s
#7 Loss: 0.1100 Acc: 95.8685% Time: 451.5314s
#8 Loss: 0.0918 Acc: 96.4710% Time: 508.1563s
#9 Loss: 0.0841 Acc: 96.7292% Time: 564.5884s
#10 Loss: 0.0894 Acc: 96.6604% Time: 620.2999s
#11 Loss: 0.0770 Acc: 96.8325% Time: 676.7584s
#12 Loss: 0.0601 Acc: 97.6588% Time: 733.8120s
#13 Loss: 0.0619 Acc: 97.8137% Time: 790.3895s
#14 Loss: 0.0593 Acc: 97.5039% Time: 846.3713s
#15 Loss: 0.0501 Acc: 97.7793% Time: 903.0465s
#16 Loss: 0.0545 Acc: 97.9515% Time: 958.6985s
#17 Loss: 0.0484 Acc: 98.0720% Time: 1014.0998s
#18 Loss: 0.0447 Acc: 98.2441% Time: 1069.9503s
#19 Loss: 0.0390 Acc: 98.5195% Time: 1125.8523s
#20 Loss: 0.0468 A

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[Test Phase] Loss: 0.6182 Acc: 90.4910% Time: 12.4935s
현재 Best는 batch_size : 8 / Loss : 0.6181861088365223 / Acc : 90.49095153808594
-------------현재 set_10 batch_szie 9 -------------
#0 Loss: 0.3463 Acc: 85.4364% Time: 56.7939s
#1 Loss: 0.2408 Acc: 89.9983% Time: 113.0835s
#2 Loss: 0.2083 Acc: 91.5820% Time: 168.9163s
#3 Loss: 0.1680 Acc: 93.3896% Time: 224.5629s
#4 Loss: 0.1488 Acc: 93.9060% Time: 280.1678s
#5 Loss: 0.1246 Acc: 94.9045% Time: 337.1754s
#6 Loss: 0.1163 Acc: 95.5414% Time: 392.7920s
#7 Loss: 0.1013 Acc: 95.8685

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[Test Phase] Loss: 0.5440 Acc: 90.3359% Time: 12.4616s
현재 Best는 batch_size : 9 / Loss : 0.544005952196349 / Acc : 90.3359146118164
-------------현재 set_10 batch_szie 10 -------------
#0 Loss: 0.3419 Acc: 85.7635% Time: 54.9695s
#1 Loss: 0.2656 Acc: 88.8449% Time: 112.9695s
#2 Loss: 0.2058 Acc: 91.5992% Time: 169.5403s
#3 Loss: 0.1704 Acc: 93.4240% Time: 225.7383s
#4 Loss: 0.1395 Acc: 94.4224% Time: 281.6253s
#5 Loss: 0.1206 Acc: 95.1971% Time: 338.2051s
#6 Loss: 0.1055 Acc: 95.7308% Time: 394.6605s
#7 Loss: 0.1011 Acc: 96.0751% Time: 451.1886s
#8 Loss: 0.0930 Acc: 96.3677% Time: 506.6076s
#9 Loss: 0.0849 Acc: 96.7981% Time: 561.5

#2 Loss: 0.1982 Acc: 91.6337% Time: 163.5644s
#3 Loss: 0.1672 Acc: 93.5273% Time: 218.3230s
#4 Loss: 0.1449 Acc: 94.1298% Time: 272.6370s
#5 Loss: 0.1197 Acc: 94.9217% Time: 327.3614s
#6 Loss: 0.1057 Acc: 96.0751% Time: 383.3364s
#7 Loss: 0.0930 Acc: 96.4366% Time: 437.9858s
#8 Loss: 0.0816 Acc: 96.9358% Time: 492.4478s
#9 Loss: 0.0701 Acc: 97.1079% Time: 546.5614s
#10 Loss: 0.0771 Acc: 97.0735% Time: 600.8126s
#11 Loss: 0.0645 Acc: 97.5039% Time: 655.0097s
#12 Loss: 0.0656 Acc: 97.2801% Time: 709.8741s
#13 Loss: 0.0552 Acc: 97.8998% Time: 764.0055s
#14 Loss: 0.0500 Acc: 98.0203% Time: 818.7834s
#15 Loss: 0.0454 Acc: 98.1408% Time: 873.1931s
#16 Loss: 0.0375 Acc: 98.4851% Time: 927.0170s
#17 Loss: 0.0378 Acc: 98.4851% Time: 981.4413s
#18 Loss: 0.0413 Acc: 98.2441% Time: 1036.6649s
#19 Loss: 0.0356 Acc: 98.4507% Time: 1091.1505s
#20 Loss: 0.0267 Acc: 98.8294% Time: 1146.1137s
#21 Loss: 0.0275 Acc: 98.7261% Time: 1200.9067s
#22 Loss: 0.0336 Acc: 98.4679% Time: 1255.4480s
#23 Loss: 0.0263

In [ ]:
Loss_Accuracy

[[10, 8, 0.6181861088365223, tensor(90.4910, device='cuda:0')],
 [10, 9, 0.544005952196349, tensor(90.3359, device='cuda:0')],
 [10, 10, 0.622033536759482, tensor(90.6977, device='cuda:0')],
 [10, 11, 0.5388996171484529, tensor(91.4212, device='cuda:0')]]

## 학습 2 후기
- 최고 성능 : batch_size = 11, Loss = 0.5388, Acc : 91.42%

# 학습 3
- 이미지 통합 set 10 활용
- batch_size는 12부터 19까지 반복

In [ ]:
Loss_Accuracy = []

In [ ]:
for i in range(10, 11):
    # set 9에서 j 번 째 학습을 의미
    for batch in range(12, 20):
            # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
            # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
        transforms_train = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
            ])

        transforms_test = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

        data_dir = '/home/tutor/안재형/dog_pic/통합'
        data_folder = os.path.join(data_dir, f'set_{i}')
        train_datasets = datasets.ImageFolder(os.path.join(data_folder, 'train'), transforms_train)
        test_datasets = datasets.ImageFolder(os.path.join(data_folder, 'test'), transforms_test)

        train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=True, num_workers=2)
        test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch, shuffle=True, num_workers=2)

    #         print('학습 데이터셋 크기:', len(train_datasets))
    #         print('테스트 데이터셋 크기:', len(test_datasets))

        class_names = train_datasets.classes
    #         print('클래스:', class_names)

            #--------------------------------------------#

        model = models.resnet34(pretrained=True)
        num_features = model.fc.in_features
            # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
        model.fc = nn.Linear(num_features, 2)
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

        #--------------------------------------------#
        num_epochs = 50
        print(f'-------------현재 set_{i} batch_szie {batch} -------------')
        model.train()
        start_time = time.time()

        # 전체 반복(epoch) 수 만큼 반복하며
        for epoch in range(num_epochs):
            running_loss = 0.
            running_corrects = 0

            # 배치 단위로 학습 데이터 불러오기
            for inputs, labels in train_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 모델에 입력(forward)하고 결과 계산
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(train_datasets)
            epoch_acc = running_corrects / len(train_datasets) * 100.

            # 학습 과정 중에 결과 출력
            print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

        #--------------------------------------------#

        model.eval()
        start_time = time.time()

        with torch.no_grad():
            running_loss = 0.
            running_corrects = 0

            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
                print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        #             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

            epoch_loss = running_loss / len(test_datasets)
            epoch_acc = running_corrects / len(test_datasets) * 100.
            print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

            Loss_Accuracy.append([i, batch, epoch_loss, epoch_acc])
            torch.save(model, f'total_set{i}_B{batch}_3.pth')
            # 모든 모델을 저장하지 않고 가장 좋은 성능의 모델을 저장
        if batch == 12 :
            Best = [i, batch, epoch_loss, epoch_acc]

        elif epoch_loss < Best[2]:
            # Best 모델보다 Loss가 개선되면 저장
            Best = [i, batch, epoch_loss, epoch_acc]

        print(f'현재 Best는 batch_size : {Best[1]} / Loss : {Best[2]} / Acc : {Best[3]}')
    #         print('--------------------------------------------------')

-------------현재 set_10 batch_szie 12 -------------
#0 Loss: 0.3100 Acc: 86.6242% Time: 41.8098s
#1 Loss: 0.2030 Acc: 91.4960% Time: 83.7581s
#2 Loss: 0.1547 Acc: 93.6134% Time: 125.5952s
#3 Loss: 0.1184 Acc: 95.3176% Time: 167.3178s
#4 Loss: 0.0934 Acc: 96.1611% Time: 208.7699s
#5 Loss: 0.0801 Acc: 97.1079% Time: 251.1987s
#6 Loss: 0.0681 Acc: 97.2629% Time: 293.1470s
#7 Loss: 0.0581 Acc: 97.7277% Time: 334.8041s
#8 Loss: 0.0482 Acc: 97.8826% Time: 376.5828s
#9 Loss: 0.0430 Acc: 98.3646% Time: 418.2134s
#10 Loss: 0.0444 Acc: 98.3302% Time: 459.0432s
#11 Loss: 0.0359 Acc: 98.3818% Time: 500.6920s
#12 Loss: 0.0337 Acc: 98.5884% Time: 543.2450s
#13 Loss: 0.0345 Acc: 98.2613% Time: 584.1530s
#14 Loss: 0.0273 Acc: 98.6400% Time: 626.5114s
#15 Loss: 0.0255 Acc: 98.7605% Time: 668.5475s
#16 Loss: 0.0260 Acc: 98.7950% Time: 709.8893s
#17 Loss: 0.0252 Acc: 98.6745% Time: 751.2540s
#18 Loss: 0.0299 Acc: 98.5195% Time: 792.8920s
#19 Loss: 0.0248 Acc: 98.7950% Time: 834.6363s
#20 Loss: 0.0242 Acc:

#30 Loss: 0.0195 Acc: 98.6745% Time: 1280.5748s
#31 Loss: 0.0192 Acc: 98.7433% Time: 1322.4812s
#32 Loss: 0.0223 Acc: 98.8294% Time: 1363.7592s
#33 Loss: 0.0186 Acc: 98.5884% Time: 1403.8773s
#34 Loss: 0.0172 Acc: 98.8638% Time: 1444.7631s
#35 Loss: 0.0158 Acc: 99.0360% Time: 1486.0710s
#36 Loss: 0.0189 Acc: 98.8638% Time: 1529.7484s
#37 Loss: 0.0215 Acc: 98.7950% Time: 1571.6164s
#38 Loss: 0.0188 Acc: 98.7089% Time: 1612.5631s
#39 Loss: 0.0175 Acc: 98.8983% Time: 1654.1642s
#40 Loss: 0.0180 Acc: 98.6573% Time: 1694.9351s
#41 Loss: 0.0175 Acc: 98.7089% Time: 1735.8151s
#42 Loss: 0.0177 Acc: 98.7089% Time: 1777.8664s
#43 Loss: 0.0184 Acc: 98.7778% Time: 1819.4185s
#44 Loss: 0.0238 Acc: 98.6917% Time: 1861.0282s
#45 Loss: 0.0253 Acc: 98.7433% Time: 1902.9035s
#46 Loss: 0.0192 Acc: 98.6573% Time: 1944.6194s
#47 Loss: 0.0211 Acc: 98.7950% Time: 1985.9957s
#48 Loss: 0.0163 Acc: 98.9155% Time: 2026.9914s
#49 Loss: 0.0181 Acc: 98.8466% Time: 2068.1414s
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (

KeyboardInterrupt: 

In [ ]:
Loss_Accuracy

[[10, 12, 0.510861769597498, tensor(90.5426, device='cuda:0')],
 [10, 13, 0.575196354101663, tensor(89.9742, device='cuda:0')]]

## 학습 3 후기
- 최고 성능 : batch_size = 12, Loss = 0.5108, Acc : 90.54%

# 통합 자료의 한계점
- 견종별 특성을 극복하기 어려운 것으로 해석된다.
    - 예를들어 웰시코기와 리트리버는 그 생김새가 정반대라서 이미지가 풍부해야 성능이 개선될 것 같다.

# 향후 개선사항
- 정상/비만 분류에 불필요한 부위(다리, 목, 머리 등)를 없애는 방법을 생각해보자
    - 몸통의 윤곽만 가져오는 방법도 있을 것 같다.